<a href="https://colab.research.google.com/github/Murakami-Shoya/2021_school/blob/master/2021_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import numpy as np
import pandas as pd
import glob

In [12]:
# パスは変更のこと
path = "/content/drive/MyDrive/leap_dataset/hand_data_2021_test/"

## データをすべてまとめる関数を作る

In [13]:
def set_data(gesture_name):

    # フレーム数でデータをスライスする量が変わる
    frame_num = 50

    output_df = pd.DataFrame()

    all_files = glob.glob(path + gesture_name + "*.csv")

    for one_file in all_files:
        one_df = pd.read_csv(one_file, sep=',')[:frame_num]
        output_df = pd.concat([output_df, one_df])

    # output_df.to_csv("./all_data_folder/all_cut_data.csv",header=False,  index=False)
    return output_df

In [15]:
set_data('glab')

,palm_position_x,palm_position_y,palm_position_z,normal_x,normal_y,normal_z,direction_x,direction_y,direction_z,T0_x,T0_y,T0_z,T1_x,T1_y,T1_z,T2_x,T2_y,T2_z,T3_x,T3_y,T3_z,I0_x,I0_y,I0_z,I1_x,I1_y,I1_z,I2_x,I2_y,I2_z,I3_x,I3_y,I3_z,I4_x,I4_y,I4_z,M0_x,M0_y,M0_z,M1_x,...,M1_z,M2_x,M2_y,M2_z,M3_x,M3_y,M3_z,M4_x,M4_y,M4_z,R0_x,R0_y,R0_z,R1_x,R1_y,R1_z,R2_x,R2_y,R2_z,R3_x,R3_y,R3_z,R4_x,R4_y,R4_z,P0_x,P0_y,P0_z,P1_x,P1_y,P1_z,P2_x,P2_y,P2_z,P3_x,P3_y,P3_z,P4_x,P4_y,P4_z
0,0.391723,0.513985,0.932136,0.054876,-0.778206,-0.625607,0.392011,0.593035,-0.703304,0.216018,0.366140,1.130307,0.214128,0.424271,0.805397,0.206472,0.466247,0.584860,0.170114,0.504354,0.450480,0.260772,0.450713,1.180595,0.335149,0.606148,0.765068,0.386426,0.623680,0.487388,0.400705,0.563096,0.357236,0.400171,0.497562,0.307395,0.311507,0.452799,1.200816,0.420804,...,0.824009,0.508704,0.625996,0.533702,0.526681,0.570106,0.365466,0.518325,0.507109,0.288402,0.361088,0.441198,1.219278,0.495368,0.558767,0.904654,0.592340,0.584746,0.647953,0.613805,0.535535,0.481084,0.604952,0.478267,0.394606,0.405322,0.400136,1.225920,0.558212,0.506947,0.969327,0.662937,0.521784,0.799294,0.695482,0.500839,0.682133,0.701331,0.465136,0.580686
1,0.390540,0.512119,0.936815,0.059246,-0.775404,-0.628679,0.394782,0.596643,-0.698687,0.214410,0.363484,1.132513,0.215278,0.421282,0.807441,0.210325,0.462744,0.586462,0.177940,0.500024,0.448955,0.258877,0.447983,1.183752,0.334377,0.604656,0.769920,0.386492,0.622813,0.492738,0.401688,0.563848,0.360974,0.401994,0.499267,0.308069,0.309542,0.450087,1.204406,0.419856,...,0.829485,0.508329,0.625086,0.539793,0.527660,0.571014,0.370434,0.520647,0.509204,0.290642,0.359079,0.438507,1.223196,0.494209,0.557124,0.910498,0.591537,0.582824,0.654068,0.614191,0.534818,0.486708,0.606675,0.478556,0.398257,0.403351,0.397491,1.229920,0.556917,0.505218,0.975331,0.661989,0.520208,0.805876,0.694837,0.499474,0.688835,0.701276,0.464160,0.587133
2,0.390531,0.510092,0.939284,0.061443,-0.773967,-0.630238,0.397287,0.598220,-0.695914,0.213959,0.361186,1.133443,0.217327,0.418118,0.808025,0.214284,0.458929,0.586651,0.182136,0.496114,0.448939,0.258315,0.445609,1.185240,0.334744,0.602872,0.772409,0.387667,0.621119,0.495631,0.403995,0.564750,0.361354,0.405524,0.501919,0.303402,0.308934,0.447680,1.206188,0.420090,...,0.832414,0.508659,0.622948,0.542722,0.529438,0.571125,0.372021,0.524139,0.511025,0.288662,0.358428,0.436070,1.225216,0.494259,0.555130,0.913716,0.591455,0.579631,0.656862,0.615213,0.532839,0.489019,0.609214,0.477805,0.398344,0.402683,0.395042,1.232041,0.556820,0.503128,0.978714,0.661967,0.517393,0.809218,0.694426,0.496084,0.692164,0.700618,0.460551,0.590616
3,0.392103,0.508271,0.940390,0.062865,-0.773658,-0.630477,0.399592,0.598393,-0.694444,0.215105,0.359389,1.133610,0.221060,0.414664,0.807563,0.220059,0.454259,0.585583,0.186382,0.491484,0.448827,0.259410,0.443772,1.185683,0.336665,0.601136,0.773340,0.390465,0.619155,0.496953,0.407905,0.565294,0.360440,0.410639,0.504322,0.297759,0.309992,0.445810,1.206865,0.421896,...,0.833725,0.510488,0.621092,0.544035,0.532820,0.571801,0.371937,0.529384,0.513650,0.284906,0.359445,0.434169,1.226110,0.495900,0.553265,0.915337,0.592879,0.576546,0.657988,0.617319,0.530412,0.489930,0.612347,0.476185,0.397876,0.403667,0.393116,1.233102,0.558316,0.501209,0.980576,0.663590,0.514883,0.811156,0.696057,0.493326,0.694223,0.702424,0.457787,0.592708
4,0.394270,0.506858,0.940471,0.062538,-0.773214,-0.631054,0.402772,0.598069,-0.692884,0.216608,0.358294,1.132768,0.224342,0.412650,0.806424,0.224904,0.451511,0.584111,0.192091,0.488452,0.446629,0.261038,0.442538,1.185143,0.339319,0.599957,0.773344,0.394139,0.617822,0.497441,0.412409,0.565836,0.359368,0.415943,0.506335,0.293297,0.311591,0.444439,1.206527,0.424425,...,0.834046,0.513180,0.619323,0.544385,0.536009,0.570872,0.371848,0.533160,0.513470,0.283514,0.360986,0.432666,1.225949,0.498215,0.551612,0.915896,0.595139,0.573892,0.658270,0.619998,0.528216,0.490051,0.615659,0.474605,0.397008,0.405096,0.391501,1

### 手のひら座標に変換する関数


引数はndarray
返り値もndarray

In [ ]:
def normalize(X):
            
    hands = X.reshape([-1, 27, 3])
    joints = []

    for hand in hands:
        palm_position = hand[0]
        direction = hand[1]
        palm_y = hand[2]
    
        palm_x = np.cross(direction, palm_y,)

        R = np.stack([palm_x, palm_y, direction])
        R = np.linalg.inv(R)

        joint = hand[3:]
        joint = joint - palm_position

        joint = np.dot(joint, R)

        palm_position = np.dot(np.array([0.0, 0.0, 0.0]), R)
        direction = np.dot(direction, R)
        palm_y = np.dot(palm_y, R)

        joint = np.vstack([palm_position, direction, palm_y, joint])

        joints.append(joint)

    joints = np.array(joints).reshape([-1, 27, 3])

    return joints

## load_hand

In [ ]:
def load_hand_data(file_name, index, train_n=800):
    output_size = len(file_name)

    train_x = []
    train_t = []
    test_x = []
    test_t = []

    for i in range(output_size):
        data = pd.read_csv(my_path + "\\my_dataset\\hand_data\\" + file_name[i], sep=',', header=None)
        data = data.values.reshape([-1, 27, 3])
        data = data[:1000]  #データを1000枚取得

        n, _, _ = data.shape
        teacher = np.array([[1 if k == index[i] else 0 for k in range(output_size)] for _ in range(n)]) #教師データの生成(ワンホットベクトル)


        data = data.reshape([-1, 100, 27*3])
        teacher = teacher.reshape([-1, 100, output_size])
        idx = np.random.permutation(np.arange(10))  #100単位をシャッフル
        data = data[idx]
        teacher = teacher[idx]
        data = data.reshape([-1, 27, 3])
        teacher = teacher.reshape([-1, output_size])


        train_data = data[:train_n]
        test_data = data[train_n:]
        train_teacher = teacher[:train_n]
        test_teacher = teacher[train_n:]

        train_x += train_data.tolist()
        test_x += test_data.tolist()

        train_t += train_teacher.tolist()
        test_t += test_teacher.tolist()

  
    #入力データは全て正規化を行う
    train_x = np.array(train_x)
    train_x = normalize(train_x)
    train_x = train_x[:, 3:]
    train_x = train_x.reshape([-1, 24*3])

    test_x = np.array(test_x)
    test_x = normalize(test_x)
    test_x = test_x[:, 3:]
    test_x = test_x.reshape([-1, 24*3])

    train_t = np.array(train_t)
    test_t = np.array(test_t)


    return train_x, train_t, test_x, test_t
